In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

class blending(object):
    
    def __init__(self, train_pred, train_y, val_pred, val_y, test_pred):
        self.train_pred = train_pred.reshape(-1,1)
        self.train_y = train_y.reshape(-1,1)
        self.val_pred = val_pred.reshape(-1,1)
        self.val_y = val_y.reshape(-1,1)
        self.test_pred = test_pred.reshape(-1,1)
        
    def train(self):
        clf = LogisticRegression(
            penalty = 'l1',
            C = 10.0,
            class_weight = 'balanced',
            random_state = 14,
            n_jobs = 72,
            solver= 'saga'
            )
        clf.fit(self.train_pred, self.train_y.ravel())
        lr_val_pred = clf.predict_proba(self.val_pred)[:, 1]
        lr_test_pred = clf.predict_proba(self.test_pred)[:, 1]
        print('Validate AUC score:', roc_auc_score(self.val_y.ravel(), lr_val_pred))
        return lr_test_pred
train_pred, train_y, val_pred, val_y, test_pred = np.random.random((7000,)), np.random.randint(0,2,size=(7000,)), np.random.random((3000,)), np.random.randint(0,2,size=(3000,)), np.random.random((3000,))
bld = blending(train_pred, train_y, val_pred, val_y, test_pred)
lr_test_pred = bld.train()

Validate AUC score: 0.5050485880487267


In [21]:
train_pred = pd.read_csv('../data/val_pred.csv')
train_y = pd.read_csv('../data/y_.csv')

In [22]:
print(train_pred.shape, train_y.shape)

(276759, 1) (276759, 1)


In [23]:
val_pred = pd.read_csv('../data/off_val_pred.csv')
val_y = pd.read_csv('../data/val_y_.csv')

In [24]:
print(val_pred.shape, val_y.shape)

(30752, 1) (30752, 1)


In [25]:
test_pred = pd.read_csv('../data/test_pred.csv')

In [26]:
train_pred, train_y, val_pred, val_y, test_pred = train_pred.values, train_y.values, val_pred.values, val_y.values, test_pred.values

In [27]:
bld = blending(train_pred, train_y, val_pred, val_y, test_pred)

In [28]:
lr_test_pred = bld.train()

Validate AUC score: 0.7499705142773484


In [29]:
lr_test_pred = pd.DataFrame(lr_test_pred)

In [30]:
sample_sub = pd.read_csv('../data/sample_submission.csv')

In [31]:
sample_sub['TARGET'] = lr_test_pred.values

In [32]:
sample_sub.head()

,SK_ID_CURR,TARGET
0,100001,0.497863
1,100005,0.586480
2,100013,0.453514
3,100028,0.458012
4,100038,0.569071


In [33]:
sample_sub.to_csv('../data/blending7499.csv', index=False)